In [1]:
import numpy as np
import helpers_and_variables as hlp
import nlp as nlp_hlp
import ML_helpers as ml_hlp
import shap
import random

import pandas as pd
# import pixiedust
# %%pixie_debugger

In [ ]:
### done this on word2vec pretrained to 7b and 63d dicts saved

# dataPath = "C:/Users/a7mad/Desktop/MEX/PekLung/PekLUng_20210503sav.xlsx"
# password = None#
# rawdataDF = hlp.get_exL_df(stringPath=dataPath, password=password, sheetNum=1)
# rawdataDF = rawdataDF.iloc[1: , :]
# dataInfoPath = "C:/Users/a7mad/Desktop/MEX/PekLung/Datainformation_minroAdjusted.xlsx"
# dataInfoDF = hlp.get_cleaned_dataInfo_df(dataInfoPath)
# katInfoDF = hlp.get_cleaned_katInfo_df(dataInfoPath)
# dict_of_katInfo = hlp.get_dict_of_katInfoDF(katInfoDF)
# dict_of_dataInfo = hlp.get_dict_of_dataInfoDF(dataInfoDF)

# rawdataDF, labels = hlp.get_labels_and_indices_unlabeled_patients(rawdataDF)

# # copy_rawdata =  hlp.get_dataframe_without_cols(rawdataDF, columns_tobe_removed=None, remove_cols_with_dates=True)
# levitsky_rawdata = hlp.get_dataframe_with_specific_cols(rawdataDF)
# del rawdataDF

# backgroud_variables = levitsky_rawdata[levitsky_rawdata.columns[0:8]]
# descriptors = levitsky_rawdata[levitsky_rawdata.columns[8:]]
# descriptors.insert(0, column=levitsky_rawdata.columns[0], value=levitsky_rawdata[levitsky_rawdata.columns[0]],
#                    allow_duplicates=False)

# ##############################################
# data_frame_to_clean = descriptors
# main_dict, ind_num_removed_features = hlp.get_dict_of_questions_answers(data_frame_to_clean, 
#                                                                         dataInfoDF, 
#                                                                         katInfoDF, 
#                                                                         amount_data =None ,
#                                                                         clear_missings_or_Non=True,
#                                                                         clear_ques_with_negative_answeres=False)
# max_num_feature_removed = 0
# ind_of_max_feature_removed = 0

# for ind, count in ind_num_removed_features:
#     if count > max_num_feature_removed:
#         max_num_feature_removed = count
#         ind_of_max_feature_removed = ind
# print("Most number of features removed from a patient is: ", max_num_feature_removed)
# print("Patient index with most removed features is: ", ind_of_max_feature_removed)
# print("Patient number or id: {0}, with remaining number of features: {1}".format(str(int(data_frame_to_clean.iloc[ind_of_max_feature_removed][0])), len(main_dict[str(int(data_frame_to_clean.iloc[ind_of_max_feature_removed][0]))])))
# print("\nTotal length of the main dictionary, number of patients: ", len(main_dict))

# #############################################
# dict_path = "C:/Users/a7mad/Desktop/MEX/PekLung/"
# hlp.write_dict_as_json_file(main_dict, dict_path)

In [ ]:
"""
read the row data as pd data frame
"""
dataPath = "C:/Users/a7mad/Desktop/MEX/PekLung/PekLUng_20210503sav.xlsx"
# password = #
rawdataDF = hlp.get_exL_df(stringPath=dataPath, password=password, sheetNum=1)
""" 
read the data info as pd data frame
"""
dataInfoPath = "C:/Users/a7mad/Desktop/MEX/PekLung/Datainformation_minroAdjusted.xlsx"
dataInfoDF = hlp.get_cleaned_dataInfo_df(dataInfoPath)
katInfoDF = hlp.get_cleaned_katInfo_df(dataInfoPath)
"""
get dictionary of data information from data info data fram
"""
dict_of_katInfo = hlp.get_dict_of_katInfoDF(katInfoDF)
dict_of_dataInfo = hlp.get_dict_of_dataInfoDF(dataInfoDF)
""" 
Get the labels, under name Lungcancer_Num
check whether labels are 1=yes LC or 2=No LC and check STUDY_1 if valid, remove unlabeled and invalid patients

"""
rawdataDF, labels, removed_indices = hlp.get_labels_and_indices_unlabeled_patients(rawdataDF)
"""
Remove features, which includes information about the label, like diagnos2, aslo modules names
DiagnosticInvestigation (need to be discussed, since it includes dignostic which means non early prediction).
columns_tobe_removed=None --> predefind columns will be removed, see the function in helpers_and_variables file.
remove_cols_with_dates=True --> removes all columns with dates(this is relevance in case of tfidf), 
otherwise consider using converting dates to days, see function hlp.get_dates_in_days() in next cell.
"""
rawdataDF =  hlp.get_dataframe_without_cols(rawdataDF, columns_tobe_removed=None, remove_cols_with_dates=False)

copy_rawdata = pd.DataFrame.copy(rawdataDF)

In [ ]:
# for nlp levitsky data
ages = levitsky_rawdata['Age']
# sns.displot(ages)
df1 = pd.DataFrame(10*np.random.randn(len(ages), 1))
df1['Age'] = levitsky_rawdata['Age'].values
# df1['Age'] 
conds = [df1.values <= 40.0 , df1.values <= 65.0,  df1.values <= 85.0 , df1.values > 85.0]
choices = ['under40Years', '40To65Years', '65To85Years', 'Over85Years']

df1 = pd.DataFrame(np.select(conds, choices, default='zero'),
             index=levitsky_rawdata.index,
             columns=df1.columns)

nlp_levitsky_rawdata = pd.DataFrame.copy(levitsky_rawdata) 
nlp_levitsky_rawdata['Age'] = df1['Age']

del df1, ages, conds, choices

nlp_backgroud_variables = nlp_levitsky_rawdata[nlp_levitsky_rawdata.columns[0:8]]
nlp_descriptors = nlp_levitsky_rawdata[nlp_levitsky_rawdata.columns[8:]]
nlp_descriptors.insert(0, column=nlp_levitsky_rawdata.columns[0],value=nlp_levitsky_rawdata[nlp_levitsky_rawdata.columns[0]], allow_duplicates=False)


In [ ]:
"""
If you wish to convert the dates into days, done only if you still have the dates columns
"""
# copy_rawdata = hlp.get_dates_in_days(copy_rawdata);

In [4]:
"""
Display the whole data frame
"""
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(katInfoDF)
# copy_rawdata

In [5]:
"""
Test different methods in hlp and extract information from the data
"""
# list_of_vars = katInfoDF['Variable'].tolist()
# list_of_labels = katInfoDF['Label'].tolist()
# list_of_values = katInfoDF['Value'].tolist()

# list_of_questions = dataInfoDF['Label'].tolist()
# list_of_quesNams = dataInfoDF['Variable'].tolist()

# cleanedListOfStrings = nlp_hlp.get_cleaned_list_of_strings(list_of_questions, stemm=True)
# cleanedListOfStrings
# cleanedWordsArray = hlp.get_array_of_words(cleanedListOfStrings)

"""
get a list with qustions and answers for one patient
"""
# list_ques_answers = list(main_dict['1001'].values())
# list_cleaned_ques_answers = nlp_hlp.get_cleaned_list_of_strings(list_ques_answers,stemm=False)
# token_words_array = hlp.get_array_of_words(list_cleaned_ques_answers)
# unique_token_words = np.unique(token_words_array, return_counts=True)


In [7]:
"""
Save to a csv or another excel file
"""
# copy_rawdata.to_csv('C:/Users/a7mad/Desktop/MEX/PekLung/dates_adjusted.csv')
# copy_rawdata.to_excel('C:/Users/a7mad/Desktop/MEX/PekLung/dates_adjusted_excel.xlsx')

# x = pd.read_csv('dates_adjusted.csv')
# x = pd.read_excel('dates_adjusted_excel.xlsx')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_dates_before)
# display(df_dates_before)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):    
#     display(df_dates_after)

In [9]:
"""
get a dictionary of each patient according to this format:
{Patient number: {questions name, the questions text, corresponding answers}}
If clear_missings_or_Non true, it clears questions (features),
where answeres are missing or the answers is no/missing.
amount_data = None --> all data is considered
"""
main_dict, ind_num_removed_features = hlp.get_dict_of_questions_answers(copy_rawdata, 
                                                                        dataInfoDF, 
                                                                        katInfoDF, 
                                                                        amount_data = None,
                                                                        clear_missings_or_Non=True,
                                                                        clear_ques_with_negative_answeres=False)
"""
check how many features were removed by get_dict_of_questions_answers()

"""
max_num_feature_removed = 0
ind_of_max_feature_removed = 0

for ind, count in ind_num_removed_features:
    if count > max_num_feature_removed:
        max_num_feature_removed = count
        ind_of_max_feature_removed = ind
print("Most number of features removed from a one patient is: ", max_num_feature_removed)
print("Patient index with most removed features is: ", ind_of_max_feature_removed)
print("Patient number or id: {0}, with remaining number of features: {1}".format(str(int(rawdataDF.loc[ind_of_max_feature_removed][0])), len(main_dict[str(int(rawdataDF.loc[ind_of_max_feature_removed][0]))])))
print("\nTotal length of the main dictionary, number of patients: ", len(main_dict))
#main_dict[str(int(rawdataDF.loc[ind_of_max_feature_removed][0]))]

Most number of features removed from a one patient is:  566
Patient index with most removed features is:  10
Patient number or id: 1013, with remaining number of features: 113

Total length of the main dictionary, number of patients:  506


In [10]:
"""
In case you need to write/save the dictionary
"""
save_dict_path = "C:/Users/a7mad/Desktop/MEX/PekLung/dict"
# save_dict_path = "C:/Users/a7mad/Desktop/MEX/PekLung/dict_nodates"
# save_dict_path = "C:/Users/a7mad/Desktop/MEX/PekLung/dict_removed_nos_nodates"
# labels_path = "C:/Users/a7mad/Desktop/MEX/PekLung/labels"

hlp.write_dict_as_json_file(main_dict, file_path=save_dict_path)
# hlp.write_list_as_json_file(labels, file_path=labels_path)
js = hlp.load_dict_from_json_file(file_path=save_dict_path)
# lbl = hlp.load_list_from_json_file(file_path=labels_path)
# print(lbl == labels)
print(js==main_dict)

True
